In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

Data Augmentation

In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest' 
)

In [3]:
x_train = train_datagen.flow_from_directory(
    r'C:\Users\monis\Downloads\chest_xray\train',
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=16,
    class_mode='categorical',
    subset='training'
)

Found 4173 images belonging to 3 classes.


In [4]:
x_val = train_datagen.flow_from_directory(
    r'C:\Users\monis\Downloads\chest_xray\train',
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=16,
    class_mode='categorical',
    subset='validation'
)

Found 1043 images belonging to 3 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [6]:
x_test = test_datagen.flow_from_directory(
    r'C:\Users\monis\Downloads\chest_xray\test',
    target_size=(256, 256),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical'
)

Found 624 images belonging to 3 classes.


Custom Model (Increased complexity)

In [14]:
model = tf.keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same', input_shape=(256, 256, 1)),
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'),
    MaxPooling2D((2, 2)),
    
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'),
    Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'),
    Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'),
    MaxPooling2D((2, 2)),
    
    Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'),
    Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    
    Dense(512, activation='relu'),
    Dropout(0.5),
    
    Dense(256, activation='relu'),
    Dropout(0.5),
    
    Dense(128, activation='relu'),
    Dropout(0.5),
    
    Dense(64, activation='relu'),
    Dropout(0.5),
    
    Dense(3, activation='softmax')  
])

In [15]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.1)

In [10]:
history = model.fit(x_train,
                    epochs=100,
                    validation_data=x_val,
                    callbacks=[early_stop, reduce_lr])

Epoch 1/100


261/261 [==============================] - 909s 3s/step - loss: 1.1984 - accuracy: 0.3817 - val_loss: 1.0737 - val_accuracy: 0.4851 - lr: 1.0000e-04
Epoch 2/100
261/261 [==============================] - 878s 3s/step - loss: 1.0671 - accuracy: 0.4654 - val_loss: 1.0301 - val_accuracy: 0.4851 - lr: 1.0000e-04
Epoch 3/100
261/261 [==============================] - 766s 3s/step - loss: 0.9969 - accuracy: 0.5200 - val_loss: 0.9292 - val_accuracy: 0.5292 - lr: 1.0000e-04
Epoch 4/100
261/261 [==============================] - 730s 3s/step - loss: 0.8934 - accuracy: 0.5840 - val_loss: 0.8382 - val_accuracy: 0.6318 - lr: 1.0000e-04
Epoch 5/100
261/261 [==============================] - 699s 3s/step - loss: 0.8188 - accuracy: 0.6259 - val_loss: 0.7942 - val_accuracy: 0.6453 - lr: 1.0000e-04
Epoch 6/100
261/261 [==============================] - 702s 3s/step - loss: 0.7762 - accuracy: 0.6489 - val_loss: 0.8100 - val_accuracy: 0.6290 - lr: 1.0000e-04
Epoch 7/100
261/261 [===========

In [11]:
test_loss, test_acc = model.evaluate(x_test)
val_loss = history.history['val_loss'][-1]
val_acc = history.history['val_accuracy'][-1]
print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)
print('Validation Loss:', val_loss)
print('Validation Accuracy:', val_acc)

20/20 [==============================] - 19s 918ms/step - loss: 0.7276 - accuracy: 0.8157
Test Loss: 0.7276424765586853
Test Accuracy: 0.8157051205635071
Validation Loss: 0.5160778164863586
Validation Accuracy: 0.7574304938316345


In [12]:
model.save('cxr_pneu1.h5')

c:\Users\monis\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
